In [1]:
%pwd

'c:\\Users\\Agent003\\PycharmProjects\\Chest-Cancer-Classification-End-to-End-using-MLflow-DVC\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Agent003\\PycharmProjects\\Chest-Cancer-Classification-End-to-End-using-MLflow-DVC'

In [5]:
# Set environment variables for MLflow tracking
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/hzain8206/Chest-Cancer-Classification-End-to-End-using-MLflow-DVC.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'hzain8206'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '23fd9f7c1c75603a0376d3ba7291d1d4d6c909c5'


In [6]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    testing_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from FineTunedClassifier.constants import *
from FineTunedClassifier.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            testing_data="artifacts/dataset/test",
            mlflow_uri="https://dagshub.com/hzain8206/Chest-Cancer-Classification-End-to-End-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [23]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _test_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            # validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            # subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._test_generator()
        self.score = model.evaluate(self.test_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-29 16:22:57,575: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-29 16:22:57,575: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-29 16:22:57,592: INFO: common: created directory at: artifacts]
Found 315 images belonging to 4 classes.
20/20 [==============================] - 201s 10s/step - loss: 9.7354 - accuracy: 0.5333
[2024-07-29 16:26:20,892: INFO: common: json file saved at: scores.json]


2024/07/29 16:26:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Agent003\AppData\Local\Temp\tmpfns5eqzs\model\data\model\assets
[2024-07-29 16:26:38,908: INFO: builder_impl: Assets written to: C:\Users\Agent003\AppData\Local\Temp\tmpfns5eqzs\model\data\model\assets]


c:\Users\Agent003\.conda\envs\chestcancerproj\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/07/29 16:28:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
